In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
#Import basic libraries
%matplotlib inline


import os
import sys
import settings
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from django_pandas.io import read_frame
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mpl_dates

from matplotlib.dates import date2num

#Import TA-lib and backtesting library
import talib
from talib.abstract import *
from talib import MA_Type

from backtesting import Strategy
from backtesting.lib import crossover

from backtesting import Backtest

/mnt/c/Users/anshu/Documents/experimental/env/lib/python3.8/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [16]:
#df = pd.read_csv('./NIFTY50_5m.csv')
#df = pd.read_csv('./NIFTY50_15m.csv')
#df = pd.read_csv('./NIFTY50_1h.csv')
df = pd.read_csv('./NIFTY50_1d.csv')
for column in df.columns:
    if column != 'date':
       df[column] = pd.to_numeric(df[column])
#df['date'] = pd.to_datetime(df['date'])
#df['date'] = df['date'].tz_localize(None)
df.set_index('date', inplace=True)
df.index = pd.to_datetime(df.index)
df.index = df.index.tz_localize(None)

df = df.sort_index()
df = df.reindex(columns = ['open', 'high', 'low', 'close', 'volume'])
df.drop('volume', axis=1, inplace=True)
df.rename(columns={"open": "Open", "high": "High", "low": "Low", "close":"Close", "volume":'Volume'}, inplace=True)

df = df[~df.index.duplicated(keep='first')]
#Optionally, filter out by date range
#print(df.head(10))
filter = True
if filter:
    start_date = '2015-01-01 09:15:00'
    end_date = '2021-12-31 03:30:00'
    df = df.loc[start_date:end_date]

df.head()

,Open,High,Low,Close
date,,,,
2015-01-02,8288.70,8410.60,8288.70,8395.45
2015-01-05,8407.95,8445.60,8363.90,8378.40
2015-01-06,8325.29,8327.85,8111.35,8127.35
2015-01-07,8118.65,8151.20,8065.45,8102.10
2015-01-08,8191.40,8243.50,8167.30,8234.60


In [20]:
from lib.pivots import getPeaks, getValleys
from lib.pivots import getHigherLows, getLowerHighs, getHigherHighs, getLowerLows

class Divergence(Strategy): #46% accuracy, poor RR
    rsi_low = 35
    rsi_high = 65
    order = 1
    
    go_long = True
    go_short = True
    def init(self):
        self.data.df['rsi'] = talib.RSI(self.data.df['Close'], 14)
        
    def is_diverging(self):
        close_highs = getPeaks(self.data.df, key='Close', order=self.order)
        close_lows = getValleys(self.data.df, key='Close', order=self.order)

        ind_highs = getPeaks(self.data.df, key='rsi', order=self.order)
        ind_lows = getValleys(self.data.df, key='rsi', order=self.order)

        if close_lows[-1] == 1 and (ind_lows[-1] == -1):
            return 1
        elif close_highs[-1] == 1 and (ind_highs[-1] == -1):
            return -1
        else:
            return 0
        
    def next(self):
        if self.go_short:
            if self.data.rsi[-1] >= self.rsi_high and self.is_diverging()==-1:
                #print('Negative divergence')
                #print(f'O: {self.data.Open[-1]} H: {self.data.High[-1]} L: {self.data.Low[-1]} C: {self.data.Close[-1]}')
                hh = getHigherHighs(self.data.Close, order=self.order)
                sl = self.data.High[-1]
                tgt = self.data.Low[-2]
                #print(f'SL: {sl} TP: {tgt}')
                if len(hh)>=1:
                    hh = [x for x in hh[-1]]
                    sl = max(self.data.Close[hh[-1]],sl)
                    #print(f'SL: {sl}')
                hl = getHigherLows(self.data.Close, order=self.order)
                if len(hl)>=1:
                    hl = [x for x in hl[-1]]
                    tgt = self.data.Close[hl[-1]]
                    #print(f'TP: {tgt}')
                    if len(hh)>=2:
                        tgt = max(tgt,self.data.Close[hh[-2]])
                        #tgt = (tgt + self.data.Close[hh[-2]])/2
                        #print(f'TP: {tgt}')
                #if self.position is None:
                if self.data.Close[-1]>tgt:
                    #print('Sell')
                    #print(f'SL: {sl} TP: {tgt}')
                    self.sell(sl=sl, tp=tgt)
        if self.go_long:
            if self.data.rsi[-1] <= self.rsi_low and self.is_diverging()==1:
                #print('Positive divergence')
                #print(f'O: {self.data.Open[-1]} H: {self.data.High[-1]} L: {self.data.Low[-1]} C: {self.data.Close[-1]}')
                ll = getLowerLows(self.data.Close, order=self.order)
                sl = self.data.Low[-1]
                tgt = self.data.High[-1]
                #print(f'SL: {sl} TP: {tgt}')
                if len(ll)>=1:
                    ll = [x for x in ll[-1]]
                    sl = min(self.data.Close[ll[-1]],sl)
                    #print(f'SL: {sl}')
                lh = getLowerHighs(self.data.Close, order=self.order)
                if len(lh)>=1:
                    lh = [x for x in lh[-1]]
                    tgt = self.data.Close[lh[-1]]
                    #print(f'TP: {tgt}')
                    if len(ll)>=2:
                        tgt = min(tgt,self.data.Close[ll[-2]])
                        #tgt = (tgt + self.data.Close[ll[-2]])/2
                        #print(f'TP: {tgt}')
                #if self.position is None:
                if self.data.Close[-1]<tgt:
                    #print('Buy')
                    #print(f'SL: {sl} TP: {tgt}')
                    self.buy(sl=sl, tp=tgt)

from backtesting import Backtest
bt = Backtest(df, Divergence, cash=100000, commission=0/1000000, trade_on_close=False)
stats = bt.run()
print(stats)

#bt.plot()

Start                     2015-01-02 00:00:00
End                       2021-12-31 00:00:00
Duration                   2555 days 00:00:00
Exposure Time [%]                    2.769763
Equity Final [$]                   102312.225
Equity Peak [$]                    112457.125
Return [%]                           2.312225
Buy & Hold Return [%]                106.7078
Return (Ann.) [%]                    0.332951
Volatility (Ann.) [%]                3.589746
Sharpe Ratio                         0.092751
Sortino Ratio                        0.155535
Calmar Ratio                         0.036908
Max. Drawdown [%]                   -9.021127
Avg. Drawdown [%]                   -3.199673
Max. Drawdown Duration      590 days 00:00:00
Avg. Drawdown Duration      309 days 00:00:00
# Trades                                   18
Win Rate [%]                        22.222222
Best Trade [%]                       6.829861
Worst Trade [%]                      -2.39299
Avg. Trade [%]                    

In [5]:
%%time

stats = bt.optimize(rsi_low=range(20, 45, 2),
                    rsi_high=range(50, 80, 2),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.rsi_high > param.rsi_low)
stats

Backtest.optimize:   0%|          | 0/13 [00:00<?, ?it/s]

/mnt/c/Users/anshu/Documents/experimental/env/lib/python3.8/site-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
/mnt/c/Users/anshu/Documents/experimental/env/lib/python3.8/site-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
/mnt/c/Users/anshu/Documents/experimental/env/lib/python3.8/site-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_tradin

CPU times: user 906 ms, sys: 281 ms, total: 1.19 s
Wall time: 54.8 s


Start                     2015-01-02 00:00:00
End                       2021-12-31 00:00:00
Duration                   2555 days 00:00:00
Exposure Time [%]                    1.269475
Equity Final [$]                    108776.25
Equity Peak [$]                     109922.85
Return [%]                            8.77625
Buy & Hold Return [%]                106.7078
Return (Ann.) [%]                    1.230764
Volatility (Ann.) [%]                2.609637
Sharpe Ratio                         0.471623
Sortino Ratio                        2.505761
Calmar Ratio                         1.179915
Max. Drawdown [%]                   -1.043095
Avg. Drawdown [%]                   -0.612099
Max. Drawdown Duration      590 days 00:00:00
Avg. Drawdown Duration      290 days 00:00:00
# Trades                                   15
Win Rate [%]                        46.666667
Best Trade [%]                       6.829861
Worst Trade [%]                     -1.000394
Avg. Trade [%]                    

In [6]:
stats._strategy

<Strategy Divergence(rsi_low=38,rsi_high=66)>

In [ ]:
from lib.pivots import getHHIndex, getLHIndex, getLLIndex, getHLIndex

class TrendFollow(Strategy):
    order = 3

    def init(self):
        pass
        
    def next(self):
        position = self.position
        hh = getHHIndex(self.data.Close, order=self.order)
        if hh is not None and len(hh)>1:
            hh = hh[-1]
        else:
            hh = -1
        hl = getHLIndex(self.data.Close, order=self.order)
        if hl is not None and len(hl)>1:
            hl = hl[-1]
        else:
            hl = -1
        ll = getLLIndex(self.data.Close, order=self.order)
        if ll is not None and len(ll)>1:
            ll = ll[-1]
        else:
            ll = -1
        lh = getLHIndex(self.data.Close, order=self.order)
        if lh is not None and len(lh)>1:
            lh = lh[-1]
        else:
            lh = -1

        if hl>ll and hl>ll:
            #Uptrend possible
            if hh > hl:
                #Uptrend confirm
                if position and position.position:
                    if position.is_short:
                        self.buy()
                else:
                    self.buy()
        if lh > hh and lh > hl:
            #downtrend possible
            if ll > lh:
                #downtrend confirm
                if position and position.position:
                    if position.is_long:
                        self.sell()
                else:
                    self.sell()

from backtesting import Backtest
bt = Backtest(df, TrendFollow, cash=100000, commission=0/1000000, trade_on_close=False, exclusive_orders=False)
stats = bt.run()
print(stats)